<a href="https://colab.research.google.com/github/Johanhms/Sql_y_Spark-Sql/blob/main/practica_avanzada_pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark==3.5.0

# Importar las bibliotecas necesarias
import os
from pyspark.sql import SparkSession

# Establecer las variables de entorno
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/usr/local/lib/python3.10/dist-packages/pyspark"

In [ ]:
import seaborn as sns
import pandas as pd
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("LeerCSV") \
    .getOrCreate()


In [ ]:
df_products = spark.read.csv('/content/Products.csv', header = True, inferSchema = True)
df_products.show(5)

+----------+-------------------+--------------------+-------------------+-------------+--------------------+
|product_id|product_category_id|        product_name|product_description|product_price|       product_image|
+----------+-------------------+--------------------+-------------------+-------------+--------------------+
|         1|                  2|Quest Q64 10 FT. ...|               NULL|        59.98|http://images.acm...|
|         2|                  2|Under Armour Men'...|               NULL|       129.99|http://images.acm...|
|         3|                  2|Under Armour Men'...|               NULL|        89.99|http://images.acm...|
|         4|                  2|Under Armour Men'...|               NULL|        89.99|http://images.acm...|
|         5|                  2|Riddell Youth Rev...|               NULL|       199.99|http://images.acm...|
+----------+-------------------+--------------------+-------------------+-------------+--------------------+
only showing top 5 

In [ ]:
df_categories = spark.read.csv('/content/categories.csv', header = True, inferSchema = True)
df_categories.show(5)

+-----------+----------------------+-------------------+
|category_id|category_department_id|      category_name|
+-----------+----------------------+-------------------+
|          1|                     2|           Football|
|          2|                     2|             Soccer|
|          3|                     2|Baseball & Softball|
|          4|                     2|         Basketball|
|          5|                     2|           Lacrosse|
+-----------+----------------------+-------------------+
only showing top 5 rows



In [ ]:
df_customer = spark.read.csv('/content/customers.csv', header = True, inferSchema = True)
df_customer.show(5)

+-----------+--------------+--------------+--------------+-----------------+--------------------+-------------+--------------+----------------+
|customer_id|customer_fname|customer_lname|customer_email|customer_password|     customer_street|customer_city|customer_state|customer_zipcode|
+-----------+--------------+--------------+--------------+-----------------+--------------------+-------------+--------------+----------------+
|          1|       Richard|     Hernandez|     XXXXXXXXX|        XXXXXXXXX|  6303 Heather Plaza|  Brownsville|            TX|           78521|
|          2|          Mary|       Barrett|     XXXXXXXXX|        XXXXXXXXX|9526 Noble Embers...|    Littleton|            CO|           80126|
|          3|           Ann|         Smith|     XXXXXXXXX|        XXXXXXXXX|3422 Blue Pioneer...|       Caguas|            PR|             725|
|          4|          Mary|         Jones|     XXXXXXXXX|        XXXXXXXXX|  8324 Little Common|   San Marcos|            CA|          

In [ ]:
df_departments = spark.read.csv('/content/departments.csv', header = True, inferSchema = True)
df_departments.show(5)

+-------------+---------------+
|department_id|department_name|
+-------------+---------------+
|            2|        Fitness|
|            3|       Footwear|
|            4|        Apparel|
|            5|           Golf|
|            6|       Outdoors|
+-------------+---------------+
only showing top 5 rows



In [ ]:
df_order_items = spark.read.csv('/content/order_items.csv', header = True, inferSchema = True)
df_order_items.show(5)

+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
|order_item_id|order_item_order_id|order_item_product_id|order_item_quantity|order_item_subtotal|order_item_product_price|
+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
|            1|                  1|                  957|                  1|             299.98|                  299.98|
|            2|                  2|                 1073|                  1|             199.99|                  199.99|
|            3|                  2|                  502|                  5|              250.0|                    50.0|
|            4|                  2|                  403|                  1|             129.99|                  129.99|
|            5|                  4|                  897|                  2|              49.98|                   24.99|
+-------------+-

In [ ]:
df_orders = spark.read.csv('/content/orders.csv', header = True, inferSchema = True)
df_orders.show(5)

+--------+-------------------+-----------------+---------------+
|order_id|         order_date|order_customer_id|   order_status|
+--------+-------------------+-----------------+---------------+
|       1|2013-07-25 00:00:00|            11599|         CLOSED|
|       2|2013-07-25 00:00:00|              256|PENDING_PAYMENT|
|       3|2013-07-25 00:00:00|            12111|       COMPLETE|
|       4|2013-07-25 00:00:00|             8827|         CLOSED|
|       5|2013-07-25 00:00:00|            11318|       COMPLETE|
+--------+-------------------+-----------------+---------------+
only showing top 5 rows



In [ ]:
df_join = df_orders.join(df_customer, df_orders.order_customer_id == df_customer.customer_id)
df_join.show(5)

+--------+-------------------+-----------------+---------------+-----------+--------------+--------------+--------------+-----------------+--------------------+-------------+--------------+----------------+
|order_id|         order_date|order_customer_id|   order_status|customer_id|customer_fname|customer_lname|customer_email|customer_password|     customer_street|customer_city|customer_state|customer_zipcode|
+--------+-------------------+-----------------+---------------+-----------+--------------+--------------+--------------+-----------------+--------------------+-------------+--------------+----------------+
|       1|2013-07-25 00:00:00|            11599|         CLOSED|      11599|          Mary|        Malone|     XXXXXXXXX|        XXXXXXXXX|8708 Indian Horse...|      Hickory|            NC|           28601|
|       2|2013-07-25 00:00:00|              256|PENDING_PAYMENT|        256|         David|     Rodriguez|     XXXXXXXXX|        XXXXXXXXX|7605 Tawny Horse ...|      Chicag

In [ ]:
# Realizar múltiples joins en una sola sentencia
df_resultados = df_order_items.join(
    df_orders,
    df_order_items.order_item_order_id == df_orders.order_id,
    "left"
).join(
    df_customer,
    df_orders.order_customer_id == df_customer.customer_id,
    "left"
).join(
    df_products,
    df_order_items.order_item_product_id == df_products.product_id,
    "left"
).join(
    df_categories,
    df_products.product_category_id == df_categories.category_id,
    "left"
).join(
    df_departments,
    df_categories.category_department_id == df_departments.department_id,
    "left"
)

# Seleccionar solo las columnas de interés para un resultado más limpio
df_final = df_resultados.select(
    "order_id",
    "order_status",
    "customer_fname",
    "customer_lname",
    "product_name",
    "product_price",
    "order_item_quantity",
    "order_item_subtotal",
    "department_name"
)

# Mostrar el resultado final
print("Resultado de la unión de múltiples tablas:")
df_final.show(truncate=False)

Resultado de la unión de múltiples tablas:
+--------+---------------+--------------+--------------+---------------------------------------------+-------------+-------------------+-------------------+---------------+
|order_id|order_status   |customer_fname|customer_lname|product_name                                 |product_price|order_item_quantity|order_item_subtotal|department_name|
+--------+---------------+--------------+--------------+---------------------------------------------+-------------+-------------------+-------------------+---------------+
|1       |CLOSED         |Mary          |Malone        |Diamondback Women's Serene Classic Comfort Bi|299.98       |1                  |299.98             |Fan Shop       |
|2       |PENDING_PAYMENT|David         |Rodriguez     |Pelican Sunstream 100 Kayak                  |199.99       |1                  |199.99             |Fan Shop       |
|2       |PENDING_PAYMENT|David         |Rodriguez     |Nike Men's Dri-FIT Victory Golf Polo

In [ ]:
from pyspark.sql.functions import *

# 1. Agrupar por cliente y categoría, y sumar los subtotales de los pedidos
df_ranking = df_resultados.groupBy("customer_fname", "customer_lname", "category_name") \
    .agg(round(sum("order_item_subtotal"),2).alias("total_comprado"))

# 2. Ordenar los resultados para crear el ranking
# Ordenamos de forma descendente (desc) para ver a los clientes con el mayor gasto primero
df_ranking_final = df_ranking.orderBy(desc("total_comprado"))

# 3. Mostrar el ranking final
print("Ranking de clientes por gasto en cada categoría:")
df_ranking_final.show(truncate=False)

Ranking de clientes por gasto en cada categoría:
+--------------+--------------+--------------------+--------------+
|customer_fname|customer_lname|category_name       |total_comprado|
+--------------+--------------+--------------------+--------------+
|Mary          |Smith         |Fishing             |827558.62     |
|Mary          |Smith         |Cleats              |537270.44     |
|Mary          |Smith         |Camping & Hiking    |502466.5      |
|Mary          |Smith         |Cardio Equipment    |439366.5      |
|Mary          |Smith         |Women's Apparel     |380150.0      |
|Mary          |Smith         |Water Sports        |375781.21     |
|Mary          |Smith         |Indoor/Outdoor Games|345411.78     |
|Mary          |Smith         |Men's Footwear      |336544.11     |
|Mary          |Smith         |Shop By Sport       |153042.92     |
|Mary          |Smith         |Electronics         |53326.46      |
|James         |Smith         |Fishing             |26398.68      |

In [ ]:
from pyspark.sql.window import Window
from pyspark.sql import functions as F

# 1. Definir la ventana
# Particionamos por cliente y categoría, y ordenamos por el subtotal del pedido
# Esto crea una "ventana" única para cada cliente en cada categoría.
window_spec = Window.partitionBy("customer_fname", "customer_lname", "category_name") \
    .orderBy(F.desc("order_item_subtotal")) # Use F.desc for ordering within the window

# 2. Aplicar la función de suma acumulada (cumulative sum)
# Calculamos el total acumulado de las compras dentro de cada partición
df_ranking_con_acumulado = df_resultados.withColumn(
    "total_acumulado_por_categoria",
    F.round(F.sum("order_item_subtotal").over(window_spec), 2) # Use F.sum and F.round
)

# 3. Mostrar el resultado
# Seleccionamos las columnas relevant es para ver el efecto de la suma acumulada
df_ranking_con_acumulado.select(
    "customer_fname",
    "customer_lname",
    "category_name",
    "order_item_subtotal",
    "total_acumulado_por_categoria"
).show()

+--------------+--------------+----------------+-------------------+-----------------------------+
|customer_fname|customer_lname|   category_name|order_item_subtotal|total_acumulado_por_categoria|
+--------------+--------------+----------------+-------------------+-----------------------------+
|         Aaron|        Berger|Camping & Hiking|             299.98|                       299.98|
|         Aaron|        Berger|     Electronics|             159.95|                       159.95|
|         Aaron|        Berger|    Water Sports|             199.99|                       399.98|
|         Aaron|        Berger|    Water Sports|             199.99|                       399.98|
|         Aaron|        Berger| Women's Apparel|              100.0|                        100.0|
|         Aaron|         Boyle|  Men's Footwear|             129.99|                       129.99|
|         Aaron|         Boyle|   Shop By Sport|              39.99|                        79.98|
|         

In [ ]:
# Agrupa por cliente y cuenta el número de pedidos
df_clientes_leales = df_resultados.groupBy("customer_fname", "customer_lname") \
    .agg(count("order_id").alias("numero_de_pedidos")) \
    .orderBy("numero_de_pedidos", ascending=False)

# Muestra el resultado
print("Top 5 clientes por número de pedidos:")
df_clientes_leales.show(5)

Top 5 clientes por número de pedidos:
+--------------+--------------+-----------------+
|customer_fname|customer_lname|numero_de_pedidos|
+--------------+--------------+-----------------+
|          Mary|         Smith|            20600|
|        Robert|         Smith|              635|
|         James|         Smith|              604|
|         David|         Smith|              589|
|          John|         Smith|              542|
+--------------+--------------+-----------------+
only showing top 5 rows



In [ ]:
# Agrupa por producto y suma las cantidades vendidas
df_productos_vendidos = df_resultados.groupBy("product_name") \
    .agg(sum("order_item_quantity").alias("total_unidades_vendidas")) \
    .orderBy(desc("total_unidades_vendidas"))

# Muestra el resultado
print("Top 5 productos más vendidos por unidades:")
df_productos_vendidos.show(5, False)

Top 5 productos más vendidos por unidades:
+---------------------------------------------+-----------------------+
|product_name                                 |total_unidades_vendidas|
+---------------------------------------------+-----------------------+
|Perfect Fitness Perfect Rip Deck             |63626                  |
|Nike Men's Dri-FIT Victory Golf Polo         |54145                  |
|O'Brien Men's Neoprene Life Vest             |50125                  |
|Nike Men's Free 5.0+ Running Shoe            |31616                  |
|Under Armour Girls' Toddler Spine Surge Runni|27262                  |
+---------------------------------------------+-----------------------+
only showing top 5 rows



In [ ]:
# Agrupa por categoría y calcula el precio promedio
df_precio_promedio = df_resultados.groupBy("category_name") \
    .agg(round(avg("product_price"),2).alias("precio_promedio")) \
    .orderBy("precio_promedio", ascending=False)

# Muestra el resultado
print("Precio promedio de los productos por categoría:")
df_precio_promedio.show(truncate=False)

Precio promedio de los productos por categoría:
+--------------------+---------------+
|category_name       |precio_promedio|
+--------------------+---------------+
|Fishing             |399.98         |
|Camping & Hiking    |299.98         |
|Water Sports        |199.99         |
|Strength Training   |189.0          |
|Golf Bags & Carts   |169.99         |
|Kids' Golf Clubs    |149.09         |
|Women's Golf Clubs  |135.61         |
|Men's Footwear      |129.99         |
|Men's Golf Clubs    |119.29         |
|As Seen on  TV!     |99.99          |
|Cardio Equipment    |97.93          |
|Soccer              |92.49          |
|Golf Shoes          |68.36          |
|Cleats              |59.99          |
|Boxing & MMA        |57.06          |
|Baseball & Softball |50.56          |
|Women's Apparel     |50.0           |
|Indoor/Outdoor Games|49.98          |
|Tennis & Racquet    |44.99          |
|Girls' Apparel      |41.75          |
+--------------------+---------------+
only showing top

In [ ]:
# 1. Crear las columnas de año y mes
# El DataFrame df_resultados ya contiene la columna 'order_date'
df_con_fecha = df_resultados.withColumn("order_year", year("order_date")) \
                          .withColumn("order_month", month("order_date"))

# 2. Agrupar por cliente, año y mes y sumar el total
df_ranking_por_mes = df_con_fecha.groupBy("order_year", "order_month") \
                               .agg(round(sum("order_item_subtotal"),2).alias("total_comprado_mes"))

# 3. Ordenar los resultados para el ranking
# Primero ordenamos por año y mes, y luego por el total de compra de forma descendente
df_ranking_final = df_ranking_por_mes.orderBy(["order_year", "order_month", desc("total_comprado_mes")])

# 4. Mostrar el ranking final
print("Ranking por total de compras por mes y año:")
df_ranking_final.show(truncate=False)

Ranking de clientes por total de compras por mes y año:
+----------+-----------+------------------+
|order_year|order_month|total_comprado_mes|
+----------+-----------+------------------+
|2013      |7          |743384.16         |
|2013      |8          |2748355.48        |
|2013      |9          |2722031.54        |
|2013      |10         |2175350.87        |
|2013      |11         |2678177.07        |
|2013      |12         |2462296.31        |
|2014      |1          |2386257.84        |
|2014      |2          |2328038.51        |
|2014      |3          |2407473.45        |
|2014      |4          |2309775.59        |
|2014      |5          |2272067.67        |
|2014      |6          |2147388.23        |
|2014      |7          |1856264.32        |
+----------+-----------+------------------+



In [ ]:
# 1. Agrupar las ventas por mes y año
df_ventas_mensuales = df_resultados.groupBy(year("order_date").alias("order_year"),
                                           month("order_date").alias("order_month")) \
                                   .agg(round(sum("order_item_subtotal"),2).alias("total_comprado_mes")) \
                                   .orderBy("order_year", "order_month")

# 2. Definir la ventana para la función lag
# La ventana particiona los datos por año y mes para un ordenamiento cronológico
window_spec = Window.orderBy("order_year", "order_month")

# 3. Aplicar lag para obtener el valor del mes anterior
df_con_mes_anterior = df_ventas_mensuales.withColumn(
    "total_mes_anterior",
    lag("total_comprado_mes", 1).over(window_spec)
)

# 4. Calcular el porcentaje de variación
# Usamos col() para referenciar las columnas y manejar los valores nulos (cuando el mes_anterior es nulo)
df_variacion_mensual = df_con_mes_anterior.withColumn(
    "variacion_porcentual",
    (round((col("total_comprado_mes") - col("total_mes_anterior")) / col("total_mes_anterior") * 100,2))
).withColumn("variacion_absoluta",
    (round((col("total_comprado_mes") - col("total_mes_anterior")),2)))

# 5. Mostrar el resultado
print("Variación porcentual mensual:")
df_variacion_mensual.show(truncate=False)

Variación porcentual mensual:
+----------+-----------+------------------+------------------+--------------------+------------------+
|order_year|order_month|total_comprado_mes|total_mes_anterior|variacion_porcentual|variacion_absoluta|
+----------+-----------+------------------+------------------+--------------------+------------------+
|2013      |7          |743384.16         |NULL              |NULL                |NULL              |
|2013      |8          |2748355.48        |743384.16         |269.71              |2004971.32        |
|2013      |9          |2722031.54        |2748355.48        |-0.96               |-26323.94         |
|2013      |10         |2175350.87        |2722031.54        |-20.08              |-546680.67        |
|2013      |11         |2678177.07        |2175350.87        |23.11               |502826.2          |
|2013      |12         |2462296.31        |2678177.07        |-8.06               |-215880.76        |
|2014      |1          |2386257.84        |

In [ ]:
# Usa 'when' para crear una columna que categoriza los pedidos
df_pedidos_categorizados = df_resultados.withColumn(
    "estado_simple",
    when(df_resultados.order_status == "COMPLETE", "Completado")
    .when(df_resultados.order_status == "PENDING", "Pendiente")
    .otherwise("Otros") # El 'otherwise' es como el 'else' y es una buena práctica
)

# Muestra los resultados para ver la nueva columna
print("Pedidos categorizados:")
df_pedidos_categorizados.select("order_id", "order_status", "estado_simple").show(10)

In [ ]:
df_resultados.select("order_status").distinct().show()

+---------------+
|   order_status|
+---------------+
|PENDING_PAYMENT|
|       COMPLETE|
|        ON_HOLD|
| PAYMENT_REVIEW|
|     PROCESSING|
|         CLOSED|
|SUSPECTED_FRAUD|
|        PENDING|
|       CANCELED|
+---------------+



In [ ]:
df_resultados.groupBy(year("order_date").alias("order_year"),month("order_date").alias("order_month"), 'order_status')\
.agg(round(sum('order_item_subtotal'),2).alias('total_comprado_por_estado'))\
.orderBy(["order_year", "order_month", desc("total_comprado_por_estado")]).show(truncate=False)

+----------+-----------+---------------+-------------------------+
|order_year|order_month|order_status   |total_comprado_por_estado|
+----------+-----------+---------------+-------------------------+
|2013      |7          |COMPLETE       |250348.39                |
|2013      |7          |PENDING_PAYMENT|167324.65                |
|2013      |7          |PROCESSING     |94902.01                 |
|2013      |7          |PENDING        |85266.17                 |
|2013      |7          |CLOSED         |76417.51                 |
|2013      |7          |ON_HOLD        |35964.82                 |
|2013      |7          |SUSPECTED_FRAUD|15830.28                 |
|2013      |7          |CANCELED       |9882.41                  |
|2013      |7          |PAYMENT_REVIEW |7447.92                  |
|2013      |8          |COMPLETE       |874680.22                |
|2013      |8          |PENDING_PAYMENT|588128.22                |
|2013      |8          |PROCESSING     |340073.52             

In [ ]:
from pyspark.sql.functions import col, when, sum, round

# Calcula el subtotal total por pedido y luego aplica la condición
df_pedidos_totales = df_resultados.groupBy("order_id") \
    .agg(round(sum("order_item_subtotal"),2).alias("total_pedido"))

# Aplica la lógica del descuento
df_con_descuento = df_pedidos_totales.withColumn(
    "monto_final",
    round(
        when(col("total_pedido") > 1000, col("total_pedido") * 0.9)
        .otherwise(col("total_pedido")),2))

# Muestra los resultados
print("Monto final de los pedidos con descuento:")
df_con_descuento.show()

Monto final de los pedidos con descuento:
+--------+------------+-----------+
|order_id|total_pedido|monto_final|
+--------+------------+-----------+
|     148|      479.99|     479.99|
|     463|      829.92|     829.92|
|     471|      169.98|     169.98|
|     496|      441.95|     441.95|
|    1088|      249.97|     249.97|
|    1580|      299.95|     299.95|
|    1591|      439.86|     439.86|
|    1645|     1509.79|    1358.81|
|    2366|      299.97|     299.97|
|    2659|      724.91|     724.91|
|    2866|      569.96|     569.96|
|    3175|      209.97|     209.97|
|    3749|      143.97|     143.97|
|    3794|      299.95|     299.95|
|    3918|      829.93|     829.93|
|    3997|      579.95|     579.95|
|    4101|      129.99|     129.99|
|    4519|       79.98|      79.98|
|    4818|      399.98|     399.98|
|    4900|      179.97|     179.97|
+--------+------------+-----------+
only showing top 20 rows

